In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
#Seleccionando columnas y países

shark_attacks = pd.read_csv('attacks.csv', sep =",", encoding = 'latin1')

shark_cols = ['Date', 'Year', 'Type', 'Country', 'Area', 'Location','Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time','Species ', 'Investigator or Source']

paises_feios = ['SPAIN']

mask = shark_attacks['Country'].isin(paises_feios)

shark_preview = shark_attacks.loc[mask&(shark_attacks['Year'] > 2000),shark_cols]

shark_preview


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
82,16-Sep-2017,2017.0,Unprovoked,SPAIN,Canary Islands,Gran Canaria,Body surfing,male,M,13,Lacerations to right foot,N,NaN,"Porbeagle, 1.5 m",Turismo La Aldea
98,26-Aug-2017,2017.0,Invalid,SPAIN,Castellón,Grao de Moncofa,Swimming,female,F,11,Lacerations to left foot,NaN,Midday,Shark involvement questionable,"El Periodico Mediterraneo, 8/27/2017"
113,23-Jul-2017,2017.0,Unprovoked,SPAIN,Majorca,"Estanys Beach, Colonia de Sant",Swimming,female,F,NaN,Abrasion to arm from shark's rough skin,N,Afternoon,Blue shark 6',"Daily Record, 7/24/2017"
130,17-Jun-2017,2017.0,Unprovoked,SPAIN,Ibiza Island,Playa den Bossa,Swimming,male,M,82,2-inch laceration to hand,N,12h00,NaN,"Diario de Ibiza, 6/17/'2017"
234,29-Jul-2016,2016.0,Unprovoked,SPAIN,Alicante Province,Arenales del Sol,Swimming,male,M,40,Lacerations to right hand,N,11h30,Blue shark,"Informacion.es, 7/29/2016"
249,Reported 08-Jul-2016,2016.0,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,10,"5 tiny puncture marks to lower leg, treated wi...",N,NaN,Angel shark,La Opinion de Tenerife
321,25-Dec-2015,2015.0,Unprovoked,SPAIN,Grand Canary Island,"Arinaga Beach, Aguimes, Gran Canaria",Swimming,Cristina Ojeda-Thies,F,38,Lacerations to left forearm,N,NaN,"Silky shark, 6.5'","Diaro de Visos, 12/26/2015"
378,18-Aug-2015,2015.0,Invalid,SPAIN,Alicante,"Poniente Beach, Benidorm",Swimming,male,M,10,Minor injury when he attempted to touch a fish.,NaN,11h00,Shark involvement not confirmed,"The Local, 8/18/2015"
471,Reported 03-Dec-2014,2014.0,Provoked,SPAIN,Granada,Off Motril,Fishing for blue sharks,male,M,NaN,Glancing bite to wrist from netted shark PROVO...,N,07h00,NaN,"ABCandalucia, 12/3/2014"
504,Sep-2014,2014.0,Unprovoked,SPAIN,Catalonia,Salou,Playing with an air mattress,male,M,16,Lacerations to right hand,N,NaN,NaN,"A. Brenneka, Shark Attack Survivors"


In [3]:
#Comprobando que no haya reduplicaciones - fecha

list_dates = shark_preview['Date'].to_list()

set_dates = set(list_dates)

len(set_dates)

16

In [4]:
#Idem

shark_preview['Date'].count()

16

In [5]:
#Seleccionando las fuentes

sources = shark_preview[['Investigator or Source']].reset_index()

In [6]:
#Pasando a lista para trabajar sobre ella

sources.drop('index', axis=1)

sources_list = sources['Investigator or Source'].to_list()

sources_list

['Turismo La Aldea',
 'El Periodico Mediterraneo, 8/27/2017',
 'Daily Record, 7/24/2017',
 "Diario de Ibiza, 6/17/'2017",
 'Informacion.es, 7/29/2016',
 'La Opinion de Tenerife',
 'Diaro de Visos, 12/26/2015',
 'The Local, 8/18/2015',
 'ABCandalucia, 12/3/2014',
 'A. Brenneka, Shark Attack Survivors',
 'Diario Informacion, 8/29/2014',
 'DiariodeAvisos, 7/25/2014',
 'DH.be, 7/11/2013',
 'gáldahora, 7/16/2010',
 'ABC-Spain, 7/25/2009',
 'C. Johansson']

In [7]:
#Primer filtro: autores

def research(x):
    pattern = '[A-Z]\. .*'
    return re.findall(pattern, x)



not_press = list(map(research, sources_list))

not_press = [x[0] for x in not_press if len(x) > 0]

not_press

['A. Brenneka, Shark Attack Survivors', 'C. Johansson']

In [8]:
#Segundo filtro: miscelánea

def strange(x):
    pattern = '(Turismo.*)|(^[a-z]á[a-z]*W*.*)'
    return re.findall(pattern, x)

strange = list(map(strange, sources_list))

strange = [k for i in strange for j in i for k in j if len(k)>0]

#Series - no prensa

for i in strange:
    not_press.append(i)

not_press = pd.Series(not_press)

not_press

0    A. Brenneka, Shark Attack Survivors
1                           C. Johansson
2                       Turismo La Aldea
3                   gáldahora, 7/16/2010
dtype: object

In [9]:
#Clasificación

shark_preview.loc[shark_preview['Investigator or Source'].isin(not_press), 'Press'] = 'not press'
shark_preview.loc[shark_preview['Press']!='not press', 'Press'] = 'press'
shark_preview

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Press
82,16-Sep-2017,2017.0,Unprovoked,SPAIN,Canary Islands,Gran Canaria,Body surfing,male,M,13,Lacerations to right foot,N,NaN,"Porbeagle, 1.5 m",Turismo La Aldea,not press
98,26-Aug-2017,2017.0,Invalid,SPAIN,Castellón,Grao de Moncofa,Swimming,female,F,11,Lacerations to left foot,NaN,Midday,Shark involvement questionable,"El Periodico Mediterraneo, 8/27/2017",press
113,23-Jul-2017,2017.0,Unprovoked,SPAIN,Majorca,"Estanys Beach, Colonia de Sant",Swimming,female,F,NaN,Abrasion to arm from shark's rough skin,N,Afternoon,Blue shark 6',"Daily Record, 7/24/2017",press
130,17-Jun-2017,2017.0,Unprovoked,SPAIN,Ibiza Island,Playa den Bossa,Swimming,male,M,82,2-inch laceration to hand,N,12h00,NaN,"Diario de Ibiza, 6/17/'2017",press
234,29-Jul-2016,2016.0,Unprovoked,SPAIN,Alicante Province,Arenales del Sol,Swimming,male,M,40,Lacerations to right hand,N,11h30,Blue shark,"Informacion.es, 7/29/2016",press
249,Reported 08-Jul-2016,2016.0,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,10,"5 tiny puncture marks to lower leg, treated wi...",N,NaN,Angel shark,La Opinion de Tenerife,press
321,25-Dec-2015,2015.0,Unprovoked,SPAIN,Grand Canary Island,"Arinaga Beach, Aguimes, Gran Canaria",Swimming,Cristina Ojeda-Thies,F,38,Lacerations to left forearm,N,NaN,"Silky shark, 6.5'","Diaro de Visos, 12/26/2015",press
378,18-Aug-2015,2015.0,Invalid,SPAIN,Alicante,"Poniente Beach, Benidorm",Swimming,male,M,10,Minor injury when he attempted to touch a fish.,NaN,11h00,Shark involvement not confirmed,"The Local, 8/18/2015",press
471,Reported 03-Dec-2014,2014.0,Provoked,SPAIN,Granada,Off Motril,Fishing for blue sharks,male,M,NaN,Glancing bite to wrist from netted shark PROVO...,N,07h00,NaN,"ABCandalucia, 12/3/2014",press
504,Sep-2014,2014.0,Unprovoked,SPAIN,Catalonia,Salou,Playing with an air mattress,male,M,16,Lacerations to right hand,N,NaN,NaN,"A. Brenneka, Shark Attack Survivors",not press


In [10]:
shark_preview.groupby(by = 'Press').count()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
Press,,,,,,,,,,,,,,,
not press,4,4,4,4,4,4,4,4,4,4,4,3,1,3,4
press,12,12,12,12,12,12,12,11,11,9,12,8,7,10,12


In [11]:
#Proporción casos en la prensa

mask = shark_preview['Press'] == 'press'

n_total_attacks = shark_preview['Date'].count()
n_attacks_press = shark_preview.loc[mask, 'Date'].count()
prop_attacks_press = n_attacks_press/n_total_attacks

prop_attacks_press

0.75

In [12]:
#Proporció casos no confirmados en prensa

unconfirmed_in_press = shark_preview.loc[mask&(shark_preview['Type'] == 'Invalid'), 'Date'].count()
prop_uncon_press = unconfirmed_in_press/n_attacks_press

prop_uncon_press

0.3333333333333333

In [13]:
#Casos confirmados letales

mask2 = (shark_preview['Type'] != 'Invalid')

shark_preview.loc[mask2&(shark_preview['Fatal (Y/N)'] == 'Y'),'Date'].count()


0